# Основные этапы обработки Landsat


1. [Предобработка данных](005_Preprocessing.ipynb).
2. Поиск рубок.
 1. Действительно ли рубки отличаются от других объектов? На примере [медиан снимков, полученных за зимний период](050_ML_exploration_basemaps.ipynb).
 2. [Поиск рубок на базовых слоях с использованим машинного обучения](050_ML_basemaps.ipynb).

## Список блокнотов с кратким описанием

Название | Описание содержимого
----------|--------------------
Intro.ipynb | Информация о том, как пользоваться блокнотами.
000_000_TableOfContext.ipynb | Данный блокнот, предназначен для сбора общей информации об остальных документах
000_Initialization.ipynb | Создание базы данных; Импорт основных и вспомогательных данных.
001_Footprints.ipynb | Создание "отпечатков" сцен.
002_TemporalDB.ipynb |  Создание пространственно-временной БД GRASS.
003_BaseMaps.ipynb | Вопрос о том, насколько базовые слои за разные годы похожи друг на друга.
005.1_CloudMasks.ipynb | Очистка сцен от облаков и их теней на базе канала BQA
005_Preprocessing.ipynb | Описание общей схемы предобработки сцен перед анализом.
006_TopoCorrection.ipynb | Топографическая коррекция сцен.
007.1_Normalization.ipynb | Нормализация зимних снимков к базовому слою, полученному медианным фильтром.
007_Normalization.ipynb | Нормализация снимков относительно друг дгуга.
010_Exploration.ipynb | Попытка понять, как выглядят рубки, можно ли использовать текстурный подход и данные PlanetLabs.
020_RasterSelection.ipynb | Определение границ зимнего периода по самим данным.
030_WinterChanges.ipynb | Рассматривается вопрос о том, как проявляются рубки на зимних снимках (сравнивается отражательная способность пикселей до рубки и после).
040_NeuralNet.ipynb | Поиск рубок на базе мнослойного перцептрона.
050_ML_basemaps.ipynb | Поиск рубок, предикторы - разность базовых слоев.
050_ML_exploration_basemaps.ipynb | Как отличаются рубки от не-рубок на базовых слоях
050_ML_exploration_basemaps.1.ipynb | Как отличаются рубки от не-рубок на базовых слоях (продолжение).
050_ML.ipynb | Поиск рубок различными моделями (предикторы - сцены без нормализации).
051_ML.norm1.ipynb | Поиск рубок различными моделями (предикторы - сцены с нормализацией).
CMASK_exploration.ipynb | Построение масок облачности и снега на базе статмоделей.
CMask_trainings.ipynb | 
NormMOD43A4.ipynb | 



# Названия растров на разных этапах обработки

Поскольку обработка состоит из большого числа этапов, то удобно все результаты того или иного преобразования сцены называть по одной схеме:

Номер |Этап | Пример названия|Документ
-----|-----------------------|-
1 | Импорт | от этого этапа остаются только LC81120282015189LGN00_BQA, остальные удалены.|[000_Initialization.ipynb](000_Initialization.ipynb)
2 | Расчет ToAR для импортированных растров из этапа 1. Все значения, выпадающие из диапазона [0, 1], считаются ошибочными и приводятся к соответствующей границе (нулю или единице). | toar_LC81120272015045LGN00_B9|[000_Initialization.ipynb](000_Initialization.ipynb)
3 | Топографическая коррекция для растров из этапа 2 | topo.toar_LC81120272015045LGN00_B6|[006_TopoCorrection.ipynb](006_TopoCorrection.ipynb)
4 | Применена маска облачности к растрам из этапа 1 (без топокоррекции). Все, что под облаками - NULL | clean.LC81130272016055LGN00_B3|
5 | Применена маска облачности к растрам из этапа 3 (c топокоррекцией). Все, что под облаками - NULL | clean.topo.toar_LC81120272015045LGN00_B6|[005.1_CloudMasks.ipynb](005.1_CloudMasks.ipynb)
6 | Нормализованы (зимние!) растры из п.5: применена маска лесов, найдены PIF, произведено нормирование к среднему 0, дисперсии 1. | norm.clean.topo.toar_LC81120272015333LGN00_B3| [007_Normalization.ipynb](007_Normalization.ipynb)


# Вспомогательные данные

## Растровые

№| Данные| Описание |Location / Mapset | Название карты | Документ
-|----|--|-----------------------|--|--
1|SRTM | Импортированы два растра высот 'srtm_63_03', 'srtm_64_03' и склеены в одно покрытие|FOREST / srtm | srtm| [006_TopoCorrection.ipynb](006_TopoCorrection.ipynb)
2| Покрытие лесами | Слой treecover  https://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.2.html (покрытие лесов) за 2000г. | FOREST / treecover| treecover| [000_Initialization.ipynb](000_Initialization.ipynb)
3a| Маска лесов | Переклассифицированный слой из п.2: все, что меньше 30% сброшено в 0, остальное в 1. | FOREST / treecover| forest.mask.30| [000_Initialization.ipynb](000_Initialization.ipynb)
3b| Маска лесов | Переклассифицированный слой из п.2: все, что меньше 60% сброшено в 0, остальное в 1. | FOREST / treecover| forest.mask.60| [000_Initialization.ipynb](000_Initialization.ipynb)
4a| Псевдоинвариантные пиксели сцены| Берется растр clean.topo.toar_xxx_Bx и второй растр того же path/row но за другую дату. Расчитываются PIF и сохраняются в растр. Затем процедура повторяется для остальных растров того же path/row. В итоге получаем множество растров потенциальных PIF для выбранной сцены| FOREST / landsat| Пример названия: pif.0.clean.topo.toar_LC81120272014074LGN00_B2| [007_Normalization.ipynb](007_Normalization.ipynb#Implementation) 
4b| Аггрегированные псевдоинвариантные пиксели сцены| Берутся растры из 4a для выбранной сцены и | FOREST / landsat| Пример названия: agg.pif.clean.topo.toar_LC81120272014074LGN00_B2| [007_Normalization.ipynb](007_Normalization.ipynb#Implementation) 
5| Базовые слои | Композитные зимние снимки, полученые медианным фильтром. Каждый слой - 13 каналов: первые 11 это исходные каналы, 12-й и 13-й каналы растра это fmask и BQA  | FOREST / basemaps| Пример названия: composite17.3 (3-й канал композита за 17-й год)| [000_Initialization.ipynb](000_Initialization.ipynb#import_landsat_composites) 
5a| Разность базовых слоев | Поканальная разность базовых слоев из п.5 | FOREST / basemaps| Пример названия: diff1615.3 (разность в 3-м канале: 16-й год минус 15-й)| [003_BaseMaps.ipynb](003_BaseMaps.ipynb) 
5b| Нормированная разность базовых слоев |Поканальная разность базовых слоев из п.5: $norm = \frac{Year_1 - Year_2}{Year_1 + Year2}$   | FOREST / basemaps| Пример названия: norm.diff1615.3 | [003_BaseMaps.ipynb](003_BaseMaps.ipynb) 
6| ALARM | Слои ALARM, распространяемые UMD|FOREST / umd_alarm | Пример названия: alarm_2014 | [000_Initialization.ipynb](000_Initialization.ipynb#import_alarm) 
6a| Зимние рубки на базе ALARM | Контура зимних рубок, построенных на базе ALARM из п.6 |FOREST / umd_alarm | Пример названия: wint1516.alrm | [000_Initialization.ipynb](000_Initialization.ipynb#import_alarm) 

## Векторные

№|Данные| Описание |Location/Mapset | Название карты|Документ
-|-----|--|-----------------------|-
1|Вырубки | Обучающие полигоны по вырубкам за 2015-2016 гг.|FOREST/PERMANENT | train15_16|[000_Initialization.ipynb](000_Initialization.ipynb)
2|Не-Вырубки | Обучающие полигоны по не-вырубкам за 2015-2016 гг.|FOREST/PERMANENT | nochange15_16|[000_Initialization.ipynb](000_Initialization.ipynb)
3a|Генерализованные ALARM| Векторизованные слои ALARM (см. раздел "Растровые данные", п. 6). Растры обработаны модальным фильтром | FOREST / umd_alarm |mod5_alarm_2014 | созданы вручную: for MAP in alarm_2014 alarm_2015 alarm_2016 ; do r.neighbors in=$MAP out=mod5_$MAP met=mode  size=5; r.null mod5_$MAP setnull=0; r.to.vect in=mod5_$MAP out=mod5_$MAP type=area col=day; v.out.ogr in=mod5_$MAP out=mod5_${MAP}.kml format=KML; done
3b| Зимние рубки на базе ALARM | Точки-пиксели зимних рубок, построенных на базе ALARM из п.6a(растры) |FOREST / umd_alarm | Пример названия: wint1516_alrm | [000_Initialization.ipynb](000_Initialization.ipynb#import_alarm) 

